In [1]:
import numpy as np
import pandas as pd

import os
import sys

# Get the current working directory (where the notebook is located)
notebook_dir = os.getcwd()

# If you need to add a specific directory relative to the notebook directory, adjust the path accordingly
desired_path = os.path.join(notebook_dir, '.')

# Add the directory to sys.path
sys.path.insert(0, desired_path)


### What we need to load/create the source Datasets with indicators and signals

In [2]:

#from jgtpy import JGTCDSSvc as cdssvc



### From bellow, we import what we will require to add MFI lags on the TTF etc

In [1]:

from jgtutils.jgtconstants import MFI_VAL,MFI_SIGNAL,VOLUME,FDB_TARGET as TARGET
#from jgtpy import mfihelper as mfih
from jgtml.mfihelper2 import get_mfi_features_column_list_by_timeframe
from jgtml import anhelper



* Create the dataset

In [4]:
i="SPX500";t="D1"


### --@STCIssue might be from ptottf import read_ttf_csv


In [5]:

from jgtml.ptottf import read_ttf_csv
#@STCIssue THAT is What we need
ttfdf=read_ttf_csv(i, t, use_full=True)

   Read TTF:  /var/lib/jgt/full/data/ttf/SPX500_D1_ttf.csv


In [6]:
ttfdf.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'tjaw', 'tteeth', 'tlips', 'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5',
       'fh8', 'fl8', 'fh13', 'fl13', 'fh21', 'fl21', 'fh34', 'fl34', 'fh55',
       'fl55', 'fh89', 'fl89', 'mfi', 'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz',
       'zlc', 'zlcb', 'zlcs', 'zcol', 'sz', 'bz', 'acs', 'acb', 'ss', 'sb',
       'price_peak_above', 'price_peak_bellow', 'ao_peak_above',
       'ao_peak_bellow', 'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake',
       'mfi_sig', 'mfi_str', 'mfi_str_M1', 'zcol_M1', 'ao_M1', 'mfi_str_W1',
       'zcol_W1', 'ao_W1'],
      dtype='object')

In [7]:
ttfdf.tail(3)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_fade,mfi_fake,mfi_sig,mfi_str,mfi_str_M1,zcol_M1,ao_M1,mfi_str_W1,zcol_W1,ao_W1
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-02 21:00:00,5513.96,5547.01,5511.19,5542.12,5514.36,5547.51,5511.79,5542.62,51015,5514.16,...,1,0,3,--,-+,gray,0.822879,-+,gray,0.803511
2024-07-03 21:00:00,5542.12,5548.70,5537.23,5540.25,5542.62,5549.30,5537.73,5540.75,12133,5542.37,...,0,1,2,-+,-+,gray,0.822879,-+,gray,0.803511
2024-07-04 21:00:00,5540.25,5577.70,5536.90,5568.90,5540.75,5578.21,5537.50,5569.50,72693,5540.50,...,0,0,1,+-,-+,gray,0.822879,-+,gray,0.803511


### Adding Lags to data we wanted

* Added lags
* ending up with probably one row that we pass to the predictor of the model we created with the MX

## --@STCGoal Dataset is feature Learning Ready


In [8]:
def mk_xbn(i,t,bnamespace,midfix,suffix=""):
    ifn=i.replace("/","-")
    pov_fn_base=ifn+"_"+t
    outbn= pov_fn_base+ "_" +bnamespace +"_"+ midfix+"_" + suffix
    outbn=outbn.replace(" ","_").replace("__","_").replace("_.",".")
    return outbn

def mk_xfn(i,t,bnamespace,midfix,suffix="",ext=".csv"):
    outbn=mk_xbn(i,t,bnamespace,midfix,suffix)
    outfn= outbn + ext
    outfn=outfn.replace(" ","_").replace("__","_").replace("_.",".")
    return outfn

def mk_xfn_from_xbn(outbn,ext=".csv"):
    outfn= outbn + ext
    outfn=outfn.replace(" ","_").replace("__","_").replace("_.",".")
    return outfn

In [9]:
# Save this to a file to observe it in other tools

xperiment_bnamespace="pto_240706pt"
bnamespace_01_ttfdf = "_ttf"
bnamespace_05_ttfdf_with_lags = bnamespace_01_ttfdf+"_lags"


In [10]:
ttfdf.tail(2)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_fade,mfi_fake,mfi_sig,mfi_str,mfi_str_M1,zcol_M1,ao_M1,mfi_str_W1,zcol_W1,ao_W1
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-03 21:00:00,5542.12,5548.7,5537.23,5540.25,5542.62,5549.30,5537.73,5540.75,12133,5542.37,...,0,1,2,-+,-+,gray,0.822879,-+,gray,0.803511
2024-07-04 21:00:00,5540.25,5577.7,5536.90,5568.90,5540.75,5578.21,5537.50,5569.50,72693,5540.50,...,0,0,1,+-,-+,gray,0.822879,-+,gray,0.803511


In [11]:

from jgtml.mxhelper import wf_get_mxdf_and_add_mfi_features_to_df as ref_wf_get_mxdf_and_mfi_str_df

from jgtml.mxhelper import _mfi_str_add_lag_as_int as add_mfi_lagging_feature_to_ttfdf

from jgtml.mfihelper2 import column_mfi_str_in_dataframe_to_id as convert_mfi_columns_from_str_to_id

ttfdf_with_mfi_feature_ready=convert_mfi_columns_from_str_to_id(ttfdf,t,inplace=False)


In [12]:

ttfdf_with_mfi_feature_ready.tail(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_fade,mfi_fake,mfi_sig,mfi_str,mfi_str_M1,zcol_M1,ao_M1,mfi_str_W1,zcol_W1,ao_W1
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-03 21:00:00,5542.12,5548.7,5537.23,5540.25,5542.62,5549.30,5537.73,5540.75,12133,5542.37,...,0,1,2,2,2,gray,0.822879,2,gray,0.803511
2024-07-04 21:00:00,5540.25,5577.7,5536.90,5568.90,5540.75,5578.21,5537.50,5569.50,72693,5540.50,...,0,0,1,1,2,gray,0.822879,2,gray,0.803511


In [13]:

proto_df_with_lags_and_feature_ready=add_mfi_lagging_feature_to_ttfdf(ttfdf_with_mfi_feature_ready,t).copy()



In [14]:


proto_df_with_lags_and_feature_ready.tail(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,mfi_str_W1_lag_1,mfi_str_W1_lag_2,mfi_str_W1_lag_3,mfi_str_W1_lag_4,mfi_str_W1_lag_5,mfi_str_lag_1,mfi_str_lag_2,mfi_str_lag_3,mfi_str_lag_4,mfi_str_lag_5
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-03 21:00:00,5542.12,5548.7,5537.23,5540.25,5542.62,5549.30,5537.73,5540.75,12133,5542.37,...,2,2,2,3,3,3,2,3,4,4
2024-07-04 21:00:00,5540.25,5577.7,5536.90,5568.90,5540.75,5578.21,5537.50,5569.50,72693,5540.50,...,2,2,2,2,3,2,3,2,3,4


In [15]:


ttfdf_with_mfi_feature_ready.columns


Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'tjaw', 'tteeth', 'tlips', 'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5',
       'fh8', 'fl8', 'fh13', 'fl13', 'fh21', 'fl21', 'fh34', 'fl34', 'fh55',
       'fl55', 'fh89', 'fl89', 'mfi', 'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz',
       'zlc', 'zlcb', 'zlcs', 'zcol', 'sz', 'bz', 'acs', 'acb', 'ss', 'sb',
       'price_peak_above', 'price_peak_bellow', 'ao_peak_above',
       'ao_peak_bellow', 'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake',
       'mfi_sig', 'mfi_str', 'mfi_str_M1', 'zcol_M1', 'ao_M1', 'mfi_str_W1',
       'zcol_W1', 'ao_W1', 'mfi_str_M1_lag_1', 'mfi_str_M1_lag_2',
       'mfi_str_M1_lag_3', 'mfi_str_M1_lag_4', 'mfi_str_M1_lag_5',
       'mfi_str_W1_lag_1', 'mfi_str_W1_lag_2', 'mfi_str_W1_lag_3',
       'mfi_str_W1_lag_4', 'mfi_str_W1_lag_5', 'mfi_str_l

In [16]:


proto_df_with_lags_and_feature_ready.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'tjaw', 'tteeth', 'tlips', 'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5',
       'fh8', 'fl8', 'fh13', 'fl13', 'fh21', 'fl21', 'fh34', 'fl34', 'fh55',
       'fl55', 'fh89', 'fl89', 'mfi', 'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz',
       'zlc', 'zlcb', 'zlcs', 'zcol', 'sz', 'bz', 'acs', 'acb', 'ss', 'sb',
       'price_peak_above', 'price_peak_bellow', 'ao_peak_above',
       'ao_peak_bellow', 'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake',
       'mfi_sig', 'mfi_str', 'mfi_str_M1', 'zcol_M1', 'ao_M1', 'mfi_str_W1',
       'zcol_W1', 'ao_W1', 'mfi_str_M1_lag_1', 'mfi_str_M1_lag_2',
       'mfi_str_M1_lag_3', 'mfi_str_M1_lag_4', 'mfi_str_M1_lag_5',
       'mfi_str_W1_lag_1', 'mfi_str_W1_lag_2', 'mfi_str_W1_lag_3',
       'mfi_str_W1_lag_4', 'mfi_str_W1_lag_5', 'mfi_str_l

# Proto part to Upgrade

## to include above correct workflow

In [17]:

xbn_01_ttfdf = mk_xbn(i,t,xperiment_bnamespace,bnamespace_01_ttfdf,"01")
xfn_01_ttfdf=mk_xfn_from_xbn(xbn_01_ttfdf)
print(xfn_01_ttfdf)

xbn_05_ttfdf_with_lags = mk_xbn(i,t,xperiment_bnamespace,bnamespace_05_ttfdf_with_lags,"05")
xfn_05_ttfdf_with_lags=mk_xfn_from_xbn(xbn_05_ttfdf_with_lags)
print(xfn_05_ttfdf_with_lags)


xbn_05_ttfdf_with_lags_int = mk_xbn(i,t,xperiment_bnamespace,bnamespace_05_ttfdf_with_lags,"05int")
xfn_05_ttfdf_with_lags_int=mk_xfn_from_xbn(xbn_05_ttfdf_with_lags_int)
print(xfn_05_ttfdf_with_lags_int)



SPX500_D1_pto_240706pt_ttf_01.csv
SPX500_D1_pto_240706pt_ttf_lags_05.csv
SPX500_D1_pto_240706pt_ttf_lags_05int.csv


In [18]:

ttfdf_with_mfi_feature_ready.to_csv(xfn_05_ttfdf_with_lags)


In [19]:
ttfdf.to_csv(xfn_01_ttfdf)

SyntaxError: keyword argument repeated: inplace (realityhelper.py, line 31)

   Read TTF:  /var/lib/jgt/full/data/ttf/SPX500_D1_ttf.csv


AttributeError: 'NoneType' object has no attribute 'columns'

## Save that datasets in HF



In [ ]:

from datasets import Dataset

import pandas as pd


repo="jgwill/xjgtml2407pub"
